In [1]:
#Make imports
# !pip install fuzzywuzzy
# !pip install python-Levenshtein
# !pip install sklearn
# !pip install pandas
# !pip install numpy
# !pip install multiprocess

import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import string
import sklearn
import numpy as np
import time
from enum import Enum


import multiprocess
import multiprocessing as mp
from multiprocessing import Pool, Process

import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english')) - set(['at', 'do', 'your', 'from', 'to', 'out', 'no', 'the'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
start = time.time()
path = './output_data/Tue Aug 18/combined_Tue Aug 18.csv'
data = pd.read_csv(path, index_col=None, header=0, engine='python' )
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  7.162816524505615 sec


In [3]:
keywords_Imp = ['stay at home' , 'do your part', 'Responsible', 
            'home', 'house', 'cancel', 'shutdown', 'postpone',
            'school closure', 'Closure', 'business closure',
            'suspension', 'quarantine', 'lockdown', 'social distance', 
            'social distancing', 'self quarantine', 'isolat', '6-feet',
            'distance', '#clubquarantine', '#quarantinelife', '#quarantineacitivites']

keywords_Ada = ['school from home' , 'learn', 'remote', 'school food service', 
            'online shopping', 'online purchase', 'online church', 'delivery',
            'drive thru', 'to go', 'take out', 'Tiktok', 'Netflix', 'telework', 
            'zoom', 'telehealth', 'telemedicine', 'work from home', 'wfh',
            'working at home', 'working remotely', 'online meeting']

keywords_Ne = ['bored' , 'lonely', 'stress', 
            'anxiety', 'scared', 'worry', 'end', 'cabin fever',
            '#sideeffectsofquarantinelife', 'tissue paper', 'toilet paper']

keywords_Sd = ['social functions' , 'gathering', 'empty streets', 
            'interaction', 'large', 'no cars', 'non-essential',
            'travel', 'unnecessary', 'crowd']

keywords_Purp = ['Flatten the curve' , 'Slow the spread', 'slow transmission', 
            'protect', 'save', '#stayhomesavelives']

keywords_Pe = ['silver lining' , 'optimistic', 'hope', 
            'bright side', 'Safe', '#togetherapart']

In [4]:
# Constants
num_cores = mp.cpu_count()
FINAL_COL_NAME = "FINAL_TEXT"

In [5]:
def keywords_cleaning(keywords_list):
    
    #Convert to lower
    for i in range(len(keywords_list)): 
        keywords_list[i] = keywords_list[i].lower()
    
    #Remove punctuations
    for i in range(len(keywords_list)):
        keywords_list[i] = keywords_list[i].translate(str.maketrans('','',string.punctuation))
    
    #More cleaning
    for i in range(len(keywords_list)):
        keywords_list[i] = keywords_list[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

    #Remove stop words
    def remove_stopwords(data):
        output_array=[]
        for sentence in data:
            temp_list=[]
            for word in sentence.split():
                if word not in stop_words:
                    temp_list.append(word)
            output_array.append(' '.join(temp_list))
        return output_array

    keywords_list=remove_stopwords(keywords_list)

    #Stemming
    ps = PorterStemmer()
    keywords_list_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_list]
    keywords_list_final = [" ".join(sentence) for sentence in keywords_list_stem]

    return keywords_list_final


In [6]:
# Enum for facets
class Facets(Enum):
    IMPLEMENTATION = "Imp"
    ADAPTATION = "Ada"
    NEGATIVE_EMOTIONS = "Ne"
    SOCIAL_DISRUPTION = "Sd"
    PURPOSE = "Purp"
    POSITIVE_EMOTION = "Pe"

In [7]:
def fuzzy_logic(row, FINAL_COL_NAME, keywords, facet):
    keyword_match, score = process.extractOne(row[FINAL_COL_NAME], keywords, scorer = fuzz.partial_ratio)
    row['final_score'] = score
    row['final_keyword_match'] = keyword_match
    return row

In [8]:
def keep_only_highest(data, high_value, facet):
    data['final_score'] = data['final_score'].astype(int)    
    data = data[data['final_score'] == 100].reset_index(drop=True)  
    data['FACET'] = facet.value 
    return data

In [9]:
def proportion(interim_data_final, interim_data):
    numerator = interim_data_final.shape[0]
    denominator = interim_data.shape[0]
    prop_val = (numerator/denominator)
    return prop_val

In [10]:
def split_dataframe(df, nums = 4): 
    chunks = list()
    num_chunks = nums
    chunk_size = len(df) // nums
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [11]:
def multiprocessing_(data_, keywords_, facet_):
    print(data_.shape)
    start = time.time()
    pool = Pool(num_cores)
    df = split_dataframe(data_, num_cores)
    data1 = pool.starmap(fuzzy_logic, [(x, FINAL_COL_NAME, keywords_, facet_) for x in df])
    pool.close()
    pool.join()
    end = time.time()
    print("time = ", end - start)
    return data1

### Keywords Cleaning

In [12]:
keywords_Imp = keywords_cleaning(keywords_Imp)
keywords_Ada = keywords_cleaning(keywords_Ada)
keywords_Ne = keywords_cleaning(keywords_Ne)
keywords_Sd = keywords_cleaning(keywords_Sd)
keywords_Purp = keywords_cleaning(keywords_Purp)
keywords_Pe = keywords_cleaning(keywords_Pe)

### Drop Rows with Empty Tweets

In [13]:
nan_value = float("NaN")
data[FINAL_COL_NAME].replace("", nan_value, inplace=True)
data.dropna(subset = [FINAL_COL_NAME], inplace=True)

data = data.head(100)

# 1_Implementation

In [14]:
#interim_Imp = multiprocessing_(data, keywords_Imp, Facets.IMPLEMENTATION)
interim_Imp = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Imp, Facets.IMPLEMENTATION))
interim_Imp_final = keep_only_highest(interim_Imp, 100, Facets.IMPLEMENTATION)

In [15]:
interim_Imp_final

,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,...,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,TEST_FLAG,RT,FINAL_TEXT,final_score,final_keyword_match,FACET
0,Tue Aug 18 19:46:26 +0000 2020,1.295809300645007e+18,"<a href=""http://twitter.com/download/android"" ...",False,NaN,1.2957933158840197e+18,67175426.0,FletchersDogs,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,fletchersdog true look like commun wont the fu...,100,social distanc,Imp
1,Tue Aug 18 19:46:27 +0000 2020,1.2958093013327951e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,drink your water pray moistur your skin drink ...,100,home,Imp
2,Tue Aug 18 19:46:28 +0000 2020,1.2958093074018345e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.295793161437098e+18,64016042.0,iamdimplekaul,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,iamdimplekaul gembing watch quit themesp mandi...,100,lockdown,Imp
3,Tue Aug 18 19:46:28 +0000 2020,1.2958093093563843e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,warn what come veteran california busi owner f...,100,lockdown,Imp
4,Tue Aug 18 19:46:29 +0000 2020,1.2958093106775447e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.2958053907944284e+18,1.1635522593991475e+18,btspopmp3,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,btspopmp3 bt lockdown bwl era the dynamit teas...,100,lockdown,Imp
5,Tue Aug 18 19:46:29 +0000 2020,1.295809310757458e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,do peopl wear mask chin also practic social di...,100,social distanc,Imp
6,Tue Aug 18 19:46:30 +0000 2020,1.295809316902056e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.2958061333959557e+18,35017255.0,MikeReiss,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,mikereiss weei video show the practic today fa...,100,social distanc,Imp
7,Tue Aug 18 19:46:30 +0000 2020,1.295809317187154e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.2957174038258235e+18,26695674.0,_LoveMTB_,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,lovemtb one polit parti the courag to speak ou...,100,quarantin,Imp
8,Tue Aug 18 19:46:30 +0000 2020,1.2958093174346875e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,first part quarantin realli video game second ...,100,hous,Imp
9,Tue Aug 18 19:46:31 +0000 2020,1.2958093179884544e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,psg herrera run midfield cl semi spoke lockdow...,100,lockdown,Imp


# 2_Adaptation

In [16]:
interim_Ada = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Ada, Facets.ADAPTATION))
interim_Ada_final = keep_only_highest(interim_Ada, 100, Facets.ADAPTATION)

In [17]:
interim_Ada_final

,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,...,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,TEST_FLAG,RT,FINAL_TEXT,final_score,final_keyword_match,FACET
0,Tue Aug 18 19:46:28 +0000 2020,1.295809307771048e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,boypussi tight he behav covid natur asian puss...,100,to go,Ada
1,Tue Aug 18 19:46:32 +0000 2020,1.2958093241833595e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,eat vat homemad hummu clear pictur work from home,100,work from home,Ada


# 3_Negative Emotion

In [18]:
interim_Ne = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Ne, Facets.NEGATIVE_EMOTIONS))
interim_Ne_final = keep_only_highest(interim_Ne, 100, Facets.NEGATIVE_EMOTIONS)

In [19]:
interim_Ne_final

,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,...,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,TEST_FLAG,RT,FINAL_TEXT,final_score,final_keyword_match,FACET
0,Tue Aug 18 19:46:27 +0000 2020,1.295809305120313e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,1.2955693930496369e+18,7.623897950372947e+17,TheRightMelissa,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,therightmelissa imagin ignor peopl like forget...,100,worri,Ne
1,Tue Aug 18 19:46:31 +0000 2020,1.295809319729074e+18,"<a href=""http://twitter.com/download/android"" ...",False,NaN,1.295808370843279e+18,9.32391502109692e+17,ajgomesta,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,ajgomesta oh do houston the 4th largest citi t...,100,end,Ne
2,Tue Aug 18 19:46:32 +0000 2020,1.2958093248545219e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,gain weight quarantin gain lost gain weight ba...,100,end,Ne
3,Tue Aug 18 19:46:33 +0000 2020,1.2958093302442107e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,good listen might help center your anxieti do ...,100,anxieti,Ne


# 4_Social Disruption

In [20]:
interim_Sd = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Sd, Facets.SOCIAL_DISRUPTION))
interim_Sd_final = keep_only_highest(interim_Sd, 100, Facets.SOCIAL_DISRUPTION)

In [21]:
interim_Sd_final

,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,...,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,TEST_FLAG,RT,FINAL_TEXT,final_score,final_keyword_match,FACET
0,Tue Aug 18 19:46:31 +0000 2020,1.295809319729074e+18,"<a href=""http://twitter.com/download/android"" ...",False,NaN,1.295808370843279e+18,9.32391502109692e+17,ajgomesta,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,ajgomesta oh do houston the 4th largest citi t...,100,larg,Sd
1,Tue Aug 18 19:46:31 +0000 2020,1.2958093219896238e+18,"<a href=""https://about.twitter.com/products/tw...",False,NaN,1.295714621614162e+18,2347049341.0,voxdotcom,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,voxdotcom presid trump took unpreced step resp...,100,travel,Sd


# 5_Purpose

In [22]:
interim_Purp = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Purp, Facets.PURPOSE))
interim_Purp_final = keep_only_highest(interim_Purp, 100, Facets.PURPOSE)

In [23]:
interim_Purp_final

,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,...,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,TEST_FLAG,RT,FINAL_TEXT,final_score,final_keyword_match,FACET
0,Tue Aug 18 19:46:32 +0000 2020,1.2958093239904827e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,cloth face mask give 10 protect no mask at giv...,100,protect,Purp
1,Tue Aug 18 19:46:34 +0000 2020,1.2958093345181286e+18,"<a href=""http://twitter.com/download/android"" ...",False,NaN,1.2958031205100012e+18,20098015.0,davidschneider,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,davidschneid tori blood hand alreadi know half...,100,protect,Purp
2,Tue Aug 18 19:46:35 +0000 2020,1.295809336279806e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,save live the right thing the measur even lack...,100,save,Purp


# 6_Positive Emotion

In [24]:
interim_Pe = data.apply(fuzzy_logic, axis = 1, args = (FINAL_COL_NAME, keywords_Pe, Facets.POSITIVE_EMOTION))
interim_Pe_final = keep_only_highest(interim_Pe, 100, Facets.POSITIVE_EMOTION)

In [25]:
interim_Pe_final

,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,...,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,TEST_FLAG,RT,FINAL_TEXT,final_score,final_keyword_match,FACET
0,Tue Aug 18 19:46:32 +0000 2020,1.295809323398918e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.2958093226019676e+18,175263641.0,petersankoff,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,precis hope start the project hard to get foot...,100,hope,Pe
1,Tue Aug 18 19:46:32 +0000 2020,1.295809324812505e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.29567571866436e+18,7.122611627637555e+17,AddressingLife,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,addressinglif got the guidelin govt sent howev...,100,safe,Pe
2,Tue Aug 18 19:46:33 +0000 2020,1.295809326846808e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,389683366.0,networkforphl,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,networkforphl new report offer polici recommen...,100,safe,Pe


### All Proportions

In [26]:
print("For IMPLEMENTATION:", proportion(interim_Imp_final, interim_Imp))
print("For ADAPTATION:", proportion(interim_Ada_final, interim_Ada))
print("For NEGATIVE EMOTIONS:", proportion(interim_Ne_final, interim_Ne))
print("For SOCIAL DISRUPTION:", proportion(interim_Sd_final, interim_Sd))
print("For PURPOSE:", proportion(interim_Purp_final, interim_Purp))
print("For POSITIVE EMOTION:", proportion(interim_Pe_final, interim_Pe))

For IMPLEMENTATION: 0.21
For ADAPTATION: 0.02
For NEGATIVE EMOTIONS: 0.04
For SOCIAL DISRUPTION: 0.02
For PURPOSE: 0.03
For POSITIVE EMOTION: 0.03


In [27]:
merged_data = pd.concat([interim_Imp_final, interim_Ada_final, interim_Ne_final, interim_Sd_final, interim_Purp_final, interim_Pe_final])
merged_data = merged_data.reset_index(drop=True)

In [28]:
merged_data

,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,...,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,TEST_FLAG,RT,FINAL_TEXT,final_score,final_keyword_match,FACET
0,Tue Aug 18 19:46:26 +0000 2020,1.295809300645007e+18,"<a href=""http://twitter.com/download/android"" ...",False,NaN,1.2957933158840197e+18,67175426.0,FletchersDogs,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,fletchersdog true look like commun wont the fu...,100,social distanc,Imp
1,Tue Aug 18 19:46:27 +0000 2020,1.2958093013327951e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,drink your water pray moistur your skin drink ...,100,home,Imp
2,Tue Aug 18 19:46:28 +0000 2020,1.2958093074018345e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.295793161437098e+18,64016042.0,iamdimplekaul,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,iamdimplekaul gembing watch quit themesp mandi...,100,lockdown,Imp
3,Tue Aug 18 19:46:28 +0000 2020,1.2958093093563843e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,warn what come veteran california busi owner f...,100,lockdown,Imp
4,Tue Aug 18 19:46:29 +0000 2020,1.2958093106775447e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.2958053907944284e+18,1.1635522593991475e+18,btspopmp3,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,btspopmp3 bt lockdown bwl era the dynamit teas...,100,lockdown,Imp
5,Tue Aug 18 19:46:29 +0000 2020,1.295809310757458e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,do peopl wear mask chin also practic social di...,100,social distanc,Imp
6,Tue Aug 18 19:46:30 +0000 2020,1.295809316902056e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.2958061333959557e+18,35017255.0,MikeReiss,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,mikereiss weei video show the practic today fa...,100,social distanc,Imp
7,Tue Aug 18 19:46:30 +0000 2020,1.295809317187154e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.2957174038258235e+18,26695674.0,_LoveMTB_,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,lovemtb one polit parti the courag to speak ou...,100,quarantin,Imp
8,Tue Aug 18 19:46:30 +0000 2020,1.2958093174346875e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,first part quarantin realli video game second ...,100,hous,Imp
9,Tue Aug 18 19:46:31 +0000 2020,1.2958093179884544e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,NaN,NaN,NaN,NaN,Y,False,psg herrera run midfield cl semi spoke lockdow...,100,lockdown,Imp


In [29]:
merged_data['user_id'].nunique()

30

In [30]:
split = pd.get_dummies(merged_data.FACET, prefix='FACET')

In [31]:
merged_data = merged_data.reset_index(drop=True).merge(split.reset_index(drop=True), left_index=True, right_index=True)

In [32]:
merged_data

,created_at,id,source,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,...,FINAL_TEXT,final_score,final_keyword_match,FACET,FACET_Ada,FACET_Imp,FACET_Ne,FACET_Pe,FACET_Purp,FACET_Sd
0,Tue Aug 18 19:46:26 +0000 2020,1.295809300645007e+18,"<a href=""http://twitter.com/download/android"" ...",False,NaN,1.2957933158840197e+18,67175426.0,FletchersDogs,en,0.0,...,fletchersdog true look like commun wont the fu...,100,social distanc,Imp,0,1,0,0,0,0
1,Tue Aug 18 19:46:27 +0000 2020,1.2958093013327951e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,drink your water pray moistur your skin drink ...,100,home,Imp,0,1,0,0,0,0
2,Tue Aug 18 19:46:28 +0000 2020,1.2958093074018345e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.295793161437098e+18,64016042.0,iamdimplekaul,en,0.0,...,iamdimplekaul gembing watch quit themesp mandi...,100,lockdown,Imp,0,1,0,0,0,0
3,Tue Aug 18 19:46:28 +0000 2020,1.2958093093563843e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,warn what come veteran california busi owner f...,100,lockdown,Imp,0,1,0,0,0,0
4,Tue Aug 18 19:46:29 +0000 2020,1.2958093106775447e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.2958053907944284e+18,1.1635522593991475e+18,btspopmp3,en,0.0,...,btspopmp3 bt lockdown bwl era the dynamit teas...,100,lockdown,Imp,0,1,0,0,0,0
5,Tue Aug 18 19:46:29 +0000 2020,1.295809310757458e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,do peopl wear mask chin also practic social di...,100,social distanc,Imp,0,1,0,0,0,0
6,Tue Aug 18 19:46:30 +0000 2020,1.295809316902056e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.2958061333959557e+18,35017255.0,MikeReiss,en,0.0,...,mikereiss weei video show the practic today fa...,100,social distanc,Imp,0,1,0,0,0,0
7,Tue Aug 18 19:46:30 +0000 2020,1.295809317187154e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,1.2957174038258235e+18,26695674.0,_LoveMTB_,en,0.0,...,lovemtb one polit parti the courag to speak ou...,100,quarantin,Imp,0,1,0,0,0,0
8,Tue Aug 18 19:46:30 +0000 2020,1.2958093174346875e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,first part quarantin realli video game second ...,100,hous,Imp,0,1,0,0,0,0
9,Tue Aug 18 19:46:31 +0000 2020,1.2958093179884544e+18,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,en,0.0,...,psg herrera run midfield cl semi spoke lockdow...,100,lockdown,Imp,0,1,0,0,0,0
